##Preparing Data

In [1]:
from modules import *
import sklearn

In [2]:
artists = pd.read_csv('data/artists.csv.gz', compression = 'gzip')
profiles = pd.read_csv('data/profiles.csv.gz', compression = 'gzip')
test = pd.read_csv('data/test.csv.gz', compression = 'gzip')
train = pd.read_csv('data/train.csv.gz', compression = 'gzip')

In [3]:
data = pd.merge(left = train, right = profiles, how = 'left', on ='user')

In [4]:
gb = data.groupby('user')
a = gb.get_group('894484ed3c451c4468b043f363a4caf6de30f9ef')
b = gb.get_group('528200ac6ac9a35857d8952a26cf6f4738960643')
pd.merge(left = a, right = b, how = 'inner', on = 'artist', )

,user_x,artist,plays_x,sex_x,age_x,country_x,user_y,plays_y,sex_y,age_y,country_y
0,894484ed3c451c4468b043f363a4caf6de30f9ef,ef43cee7-13f5-4d10-844e-879aba3d8b19,16,f,16,United States,528200ac6ac9a35857d8952a26cf6f4738960643,42,f,16,United States
1,894484ed3c451c4468b043f363a4caf6de30f9ef,35a6a353-b186-4c13-a264-d18d5e2ce853,26,f,16,United States,528200ac6ac9a35857d8952a26cf6f4738960643,50,f,16,United States
2,894484ed3c451c4468b043f363a4caf6de30f9ef,a96ac800-bfcb-412a-8a63-0a98df600700,15,f,16,United States,528200ac6ac9a35857d8952a26cf6f4738960643,47,f,16,United States


In [5]:
demog = data.copy(deep = True)

In [6]:
demog = demog.drop('user', axis = 1)
demog['age'] = demog.age.replace(np.nan, 0) 
demog['sex'] = demog.sex.replace('NaN', -1) 
demog['sex'] = [1 if i == 'm' else (0 if i == 'f' else -1) for i in demog.sex]

In [7]:
testdf = pd.merge(left = test, right = profiles, how = 'left', on ='user')

In [8]:
testdf = testdf.drop('user', axis = 1)
testdf['age'] = testdf.age.replace(np.nan, 0) 
testdf['sex'] = testdf.sex.replace('NaN', -1) 
testdf['sex'] = [1 if i == 'm' else (0 if i == 'f' else -1) for i in testdf.sex]
testdf.head()

,Id,artist,sex,age,country
0,1,4ac4e32b-bd18-402e-adad-ae00e72f8d85,0,23,United Kingdom
1,125409,88a4cd4e-5dee-4153-8462-0d5e43c75f47,0,23,United Kingdom
2,463450,2e547c75-36c1-49d0-984e-b14498c936f0,0,23,United Kingdom
3,1793322,22a40b75-affc-4e69-8884-266d087e4751,0,23,United Kingdom
4,1953458,44cf61b8-5197-448a-b82b-cef6ee89fac5,0,23,United Kingdom


##min/diff

##General algorithm
1. For sex, age, country of user, find k-nearest neighbors that that have that country and that artist.
2. average plays of k-nearest neighbors.
3. weight with prior of average plays for that artist
4. return result

In [9]:
#clf = sklearn.neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None)
clf = sklearn.neighbors.NearestNeighbors()

In [10]:
prior = demog.groupby('artist').plays.mean()

In [ ]:
gb = demog.groupby(['artist','country'])

In [ ]:
t = datetime.now()
plays = []
for i, row in testdf.iterrows():
    result = 0
    try:
        index = tuple(row[['artist', 'country']])
        df = gb.get_group(index)
        df = df.reset_index()
        dist, indices = clf.fit(df[['sex','age']]).kneighbors(row[['sex', 'age']],n_neighbors = min(5, len(df)))
        result = df.ix[indices[0]].plays.mean() #Mean of k-nearest neighbors plays
        result = .2 * prior.ix[row['artist']] + .8*result #Incorporate prior belief
    except:
        #print (i, row)
        result = prior.ix[row['artist']]
    plays.append([i, result])
    if np.random.randint(10000) == 0:
        elapsed = datetime.now() - t
        total = (elapsed / i) * len(testdf)
        print "Time Remaining:", total - elapsed

Time Remaining: 3:55:25.434937
Time Remaining: 3:52:43.630694
Time Remaining: 3:49:11.280334
Time Remaining: 3:43:56.700864
Time Remaining: 3:43:52.189164
Time Remaining: 3:43:04.313674
Time Remaining: 3:42:52.556776
Time Remaining: 3:42:54.214208
Time Remaining: 3:42:48.579628
Time Remaining: 3:42:47.438788
Time Remaining: 3:42:16.407987
Time Remaining: 3:41:35.498311
Time Remaining: 3:40:44.485616
Time Remaining:

In [14]:
result = pd.DataFrame(plays)
result = result.apply(pd.Series.round)
result.columns = ['Id', 'Plays']
result.Id = result.Id + 1
result.to_csv('submission1.csv', index = False)

In [34]:
#Trying removing prior
bias = []
for i, row in testdf.iterrows():
    bias.append(.2 * prior.ix[row['artist']])

In [37]:
bias = pd.Series(bias)
result.Plays = (result.Plays - bias)*1.25

In [40]:
result = result.apply(pd.Series.round)
result.to_csv('submission2.csv', index = False)

In [41]:
prior.head()

artist
000d90ec-d64c-48a1-b775-e726fd240e9f    196.296078
000fc734-b7e1-4a01-92d1-f544261b43f5    288.887880
0019749d-ee29-4a5f-ab17-6bfa11deb969    145.452174
0039c7ae-e1a7-4a7d-9b49-0cbc716821a6    341.798469
004e5eed-e267-46ea-b504-54526f1f377d    370.565190
Name: plays, dtype: float64

In [42]:
testdf.head()

,Id,artist,sex,age,country
0,1,4ac4e32b-bd18-402e-adad-ae00e72f8d85,0,23,United Kingdom
1,125409,88a4cd4e-5dee-4153-8462-0d5e43c75f47,0,23,United Kingdom
2,463450,2e547c75-36c1-49d0-984e-b14498c936f0,0,23,United Kingdom
3,1793322,22a40b75-affc-4e69-8884-266d087e4751,0,23,United Kingdom
4,1953458,44cf61b8-5197-448a-b82b-cef6ee89fac5,0,23,United Kingdom


In [44]:
sub3 = testdf.artist.replace(prior)

In [46]:
sub3 = pd.DataFrame(sub3).reset_index()

In [48]:
sub3.columns = ['Id','Plays']
sub3['Id'] = sub3.Id + 1

TypeError: unbound method round() must be called with Series instance as first argument (got nothing instead)

In [49]:
sub3 = sub3.apply(pd.Series.round)

In [52]:
sub3.to_csv('submission3.csv', index = False)